Ovo će biti bilježnica s praktičnom realizacijom naših rješenja

# Opće uvjetovanosti

In [24]:
def skini(url: str, datoteka: str):
    import urllib.request as rq, pathlib
    with rq.urlopen(url) as konekcija:
        pathlib.Path(datoteka).write_bytes(konekcija.read())
        
test_podaci = 'https://raw.githubusercontent.com/jbrigljevic/str_uc/main/Podaci/test.csv'
train_podaci = 'https://raw.githubusercontent.com/jbrigljevic/str_uc/main/Podaci/train.csv'
skini(test_podaci, 'test_podaci.csv')
skini(train_podaci, 'train_podaci.csv')

In [74]:
test_data = pd.read_csv('test_podaci.csv')
train_data = pd.read_csv('train_podaci.csv')

In [75]:
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


# k-nn 

In [98]:
import nltk
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import re
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

import csv

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Juraj\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\Juraj\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Juraj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Juraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Juraj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [82]:
df_train = train_data
df_test = test_data

s = stopwords.words('english')
#dodatne stop riječi
s.extend(['today', 'tomorrow', 'outside', 'out', 'there'])
ps = nltk.wordnet.WordNetLemmatizer()

for i in range(df_train.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', df_train.loc[i,'text'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    df_train.loc[i, 'text'] = review
    
X_train = df_train['text']
y_train = df_train['target']

print("df_train nakon micanja stop riječi")
print(df_train['text'][:10])

for i in range(df_test.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', df_test.loc[i,'text'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    df_test.loc[i, 'text'] = review
    
X_test = df_test['text']

print("df_test nakon micanja stop riječi")
print(df_test['text'][:10])

df_train nakon micanja stop riječi
0           deed reason earthquake may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3    people receive wildfire evacuation order calif...
4    got sent photo ruby alaska smoke wildfire pour...
5    rockyfire update california hwy closed directi...
6    flood disaster heavy rain cause flash flooding...
7                               top hill see fire wood
8    emergency evacuation happening building across...
9                           afraid tornado coming area
Name: text, dtype: object
df_test nakon micanja stop riječi
0                          happened terrible car crash
1    heard earthquake different city stay safe ever...
2    forest fire spot pond goose fleeing across str...
3                 apocalypse lighting spokane wildfire
4                   typhoon soudelor kill china taiwan
5                                   shaking earthquake
6     probably still show

In [88]:
# Builds a dictionary of features and transforms documents to feature vectors and convert our text documents to a
# matrix of token counts (CountVectorizer)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [91]:
knn = KNeighborsClassifier(n_neighbors=1)

# training our classifier ; train_data.target will be having numbers assigned for each category in train data
clf = knn.fit(X_train_tfidf, y_train)

# Input Data to predict their classes of the given categories
primjer = [X_test[0]]
# building up feature vector of our input
X_new_counts = count_vect.transform(primjer)
# We call transform instead of fit_transform because it's already been fit
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [94]:
predicted = clf.predict(X_new_tfidf)

In [97]:
# We can use Pipeline to add vectorizer -> transformer -> classifier all in a one compound classifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', knn),
])
# Fitting our train data to the pipeline
text_clf.fit(X_train, y_train)

# Predicting our test data
predicted = text_clf.predict(X_test)

1224


In [134]:
#export u csv
dict = {"id":list(df_test["id"]), "target":predicted}
df_test_export = pd.DataFrame(dict)
df_test_export.set_index("id", inplace = True)
retci = [["id", "target"]]
for i in range(len(df_test_export.index)):
    retci.append([df_test_export.index[i], predicted[i]])

np.savetxt("test_result.csv", retci, delimiter =",", fmt ='% s')